In [ ]:
import os
import pickle
import pycisTopic
import scenicplus
print(scenicplus.__file__)

In [ ]:
import pycisTopic
print(pycisTopic.__version__)
import scenicplus
print(scenicplus.__version__)

In [ ]:
import os
os.environ
print(os.environ['CONDA_DEFAULT_ENV'])

In [ ]:
import sys
import os

print("Python executable:", sys.executable)
print("Environment name:", os.path.basename(os.path.dirname(sys.executable)))
print("Python version:", sys.version)

In [ ]:
print(os.getcwd())

In [24]:
import os
out_dir = "outs"
os.makedirs(out_dir, exist_ok = True)
temp_dir = 'tmp/'
data_path = os.path.join('scenicplus_dir')
fragment_path = os.path.join(data_path, 'data/fragment/BPD_Control_EC_fragments.tsv.gz')

group_col = 'Celltype_use'
ID_use = 'BPD'
genome_use = 'human'
peak_use = 'cellranger' ##'signac' or 'sc' or 'cellranger'

if peak_use == 'sc':
    consensus_peak_path = os.path.join(out_dir, "consensus_peak_calling/consensus_regions.bed")
elif peak_use == 'signac':
    consensus_peak_path = os.path.join(out_dir, "consensus_peak_calling/consensus_regions_signac_support_ge_1.bed")
elif peak_use == 'cellranger':
    consensus_peak_path = os.path.join(out_dir, "consensus_peak_calling/cellranger.bed")


if genome_use == 'human':
    chromsize_link = 'http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes'
    path_to_blacklist="Resources/blacklist/hg38-blacklist.v2.bed"
    genome_size = 'hs'
    ucsc_genome = 'hg38'
    ensembl_set = "hsapiens_gene_ensembl" 
    genome = 'Human'
    db_path = 'Resources/Database/ENCODE/hg38_screen_v10_clust.regions_vs_motifs.rankings.feather'
elif genome_use == 'mouse':
    chromsize_link = 'http://hgdownload.cse.ucsc.edu/goldenPath/mm10/bigZips/mm10.chrom.sizes'
    path_to_blacklist="Resources/blacklist/mm10-blacklist.v2.bed"
    genome_size = 'mm'
    ucsc_genome = 'mm10'
    ensembl_set = "mmusculus_gene_ensembl" 
    genome = 'Mouse'
    db_path = 'Resources/Database/ENCODE/mm10_screen_v10_clust.regions_vs_motifs.rankings.feather'
else:
    raise ValueError(f"Unsupported genome: {genome_use}")

fragments_dict = {
    ID_use: f"{fragment_path}"
}

# Getting pseudobulk profiles from cell annotation 

In [ ]:
#### MUltiome MODE
import scanpy as sc
import pandas as pd
adata = sc.read_h5ad(f"{data_path}/data/results/h5ad.h5ad")
cell_data = adata.obs
cell_data['celltype'] = cell_data[group_col].str.replace(r'\W', '_', regex=True)
cell_data['sample_id'] = ID_use
cell_data['barcode'] = cell_data.index.values.astype(str)
print(cell_data.shape)
uniques = cell_data['celltype'].unique()
print(uniques)
cell_data
print(cell_data['celltype'].value_counts())

In [ ]:
chromsizes = pd.read_table(
    chromsize_link,
    header = None,
    names = ["Chromosome", "End"]
)
chromsizes.insert(1, "Start", 0)
chromsizes.head()

In [ ]:
from pycisTopic.pseudobulk_peak_calling import export_pseudobulk
os.makedirs(os.path.join(out_dir, "consensus_peak_calling"), exist_ok = True)
os.makedirs(os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bed_files"), exist_ok = True)
os.makedirs(os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bw_files"), exist_ok = True)

In [ ]:
bw_paths, bed_paths = export_pseudobulk(
    input_data = cell_data,
    variable = "celltype",
    sample_id_col = "sample_id",
    chromsizes = chromsizes,
    bed_path = os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bed_files"),
    bigwig_path = os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bw_files"),
    path_to_fragments = fragments_dict,
    n_cpu = 20,
    normalize_bigwig = True,
    temp_dir = temp_dir,
    split_pattern = None
)

In [ ]:
print(bw_paths)
print(bed_paths)

In [ ]:
with open(os.path.join(out_dir, "consensus_peak_calling/bw_paths.tsv"), "wt") as f:
    for v in bw_paths:
        _ = f.write(f"{v}\t{bw_paths[v]}\n")

In [ ]:
with open(os.path.join(out_dir, "consensus_peak_calling/bed_paths.tsv"), "wt") as f:
    for v in bed_paths:
        _ = f.write(f"{v}\t{bed_paths[v]}\n")

#  Inferring consensus peaks


In [ ]:
bw_paths = {}
with open(os.path.join(out_dir, "consensus_peak_calling/bw_paths.tsv")) as f:
    for line in f:
        v, p = line.strip().split("\t")
        bw_paths.update({v: p})

In [ ]:
bed_paths = {}
with open(os.path.join(out_dir, "consensus_peak_calling/bed_paths.tsv")) as f:
    for line in f:
        v, p = line.strip().split("\t")
        bed_paths.update({v: p})

In [ ]:
print(bed_paths)

In [ ]:
from pycisTopic.pseudobulk_peak_calling import peak_calling
macs_path = "macs2"

os.makedirs(os.path.join(out_dir, "consensus_peak_calling/MACS"), exist_ok = True)

narrow_peak_dict = peak_calling(
    macs_path = macs_path,
    bed_paths = bed_paths,
    outdir = os.path.join(out_dir, "consensus_peak_calling/MACS"),
    genome_size = genome_size,
    n_cpu = 8,
    input_format = 'BEDPE',
    shift = 73,
    ext_size = 146,
    keep_dup = 'all',
    q_value = 0.05,
    _temp_dir = temp_dir
)

In [ ]:
from pycisTopic.iterative_peak_calling import get_consensus_peaks
# Other param
peak_half_width=250
path_to_blacklist=path_to_blacklist
# Get consensus peaks
consensus_peaks = get_consensus_peaks(
    narrow_peaks_dict = narrow_peak_dict,
    peak_half_width = peak_half_width,
    chromsizes = chromsizes,
    path_to_blacklist = path_to_blacklist)

In [ ]:
consensus_peaks.to_bed(
    path = os.path.join(out_dir, "consensus_peak_calling/consensus_regions.bed"),
    keep =True,
    compression = 'infer',
    chain = False)

# QC

In [ ]:
import subprocess
import tempfile
from pathlib import Path
import os

In [ ]:
# JUPYTER CELL
import os
import pandas as pd
import polars as pl
from pybiomart import Dataset
from pycisTopic.gene_annotation import change_chromosome_source_in_bed, write_tss_annotation_to_bed

# Expected in session:
#   ensembl_set = "hsapiens_gene_ensembl"  # or "mmusculus_gene_ensembl"
#   ucsc_genome = "hg38"                   # or "mm10"
out_path = "outs/qc/tss.bed"

# 0) choose Ensembl host (hostname only; no https://)
if ucsc_genome == "hg38":
    host = "www.ensembl.org"                 # GRCh38
    chroms = [str(i) for i in range(1, 23)] + ["X", "Y", "MT"]
elif ucsc_genome == "mm10":
    host = "jul2020.archive.ensembl.org"      # GRCm38/mm10
    chroms = [str(i) for i in range(1, 20)] + ["X", "Y", "MT"]
else:
    raise ValueError(f"Unsupported ucsc_genome: {ucsc_genome}")

print(f"[TSS] BioMart dataset={ensembl_set}, host={host}, UCSC={ucsc_genome}")

# 1) Fetch per-transcript TSS (dtype-safe)
ds = Dataset(name=ensembl_set, host=host, use_cache=True)
df = ds.query(
    attributes=[
        "chromosome_name",
        "transcription_start_site",
        "strand",
        "external_gene_name",
        "transcript_biotype",
        "ensembl_transcript_id",  # ensure per-transcript granularity
    ],
    filters={"transcript_biotype": ["protein_coding"]},
)

# Normalize column names
df = df.rename(columns={
    "Chromosome/scaffold name": "chromosome_name",
    "Transcription start site (TSS)": "transcription_start_site",
    "External gene name": "external_gene_name",
    "Gene name": "external_gene_name",
    "Transcript type": "transcript_biotype",
    "Strand": "strand",
    "Ensembl Transcript ID": "ensembl_transcript_id",
})

# Coerce everything to string to avoid ArrowTypeError in Polars
for c in df.columns:
    df[c] = df[c].astype("string")

# Keep primary chromosomes only (match UCSC main set)
df = df[df["chromosome_name"].isin(chroms)].copy()

# 2) Build Polars BED-like frame (per-transcript, 1-bp TSS)
tss_pl = pl.from_pandas(df).select(
    [
        pl.col("chromosome_name").alias("Chromosome"),
        (pl.col("transcription_start_site").cast(pl.Int64) - 1).cast(pl.Int32).alias("Start"),
        pl.col("transcription_start_site").cast(pl.Int64).cast(pl.Int32).alias("End"),
        pl.col("external_gene_name").alias("Gene"),
        pl.lit(".").alias("Score"),
        pl.when(pl.col("strand").cast(pl.Int32) == 1)
          .then(pl.lit("+"))
          .otherwise(
              pl.when(pl.col("strand").cast(pl.Int32) == -1)
                .then(pl.lit("-"))
                .otherwise(pl.lit("."))
          ).alias("Strand"),
        pl.col("transcript_biotype").alias("Transcript_type"),
    ]
)

# 3) Build the chromosome alias table manually (Ensembl -> UCSC)
#    Ensembl names: 1..22/X/Y/MT ; UCSC: chr1..chr22/chrX/chrY/chrM
ensembl_names = chroms
ucsc_names = [("chrM" if c == "MT" else "chr" + c) for c in ensembl_names]
alias_pl = pl.DataFrame({"ensembl": ensembl_names, "ucsc": ucsc_names})

# 4) Convert names using your version's signature:
#    change_chromosome_source_in_bed(alias_table, bed_df, "ensembl", "ucsc")
tss_pl_ucsc = change_chromosome_source_in_bed(alias_pl, tss_pl, "ensembl", "ucsc")

# 5) Write BED with the exact header the CLI writes
os.makedirs(os.path.dirname(out_path), exist_ok=True)
write_tss_annotation_to_bed(tss_pl_ucsc, out_path)

print("Columns:", tss_pl_ucsc.columns)
print(tss_pl_ucsc.head())
print(f"Wrote {out_path}")


In [ ]:
# ---- Command block ----
cmd = f"""

module load anaconda3
source activate scenicplus_rhel9

#pycistopic tss gene_annotation_list | grep -i {genome}
#mkdir -p outs/qc
#pycistopic tss get_tss --output outs/qc/tss.bed --name "{ensembl_set}" --to-chrom-source ucsc --ucsc {ucsc_genome}
#head outs/qc/tss.bed | column -t
pycistopic qc --fragments {fragments_dict[ID_use]} \
              --regions {consensus_peak_path} \
              --tss outs/qc/tss.bed \
              --output outs/qc/{ID_use}
"""

# ---- Run everything in one bash shell ----
res = subprocess.run(["bash", "-lc", cmd], text=True, capture_output=True)

print(res.stdout)
if res.returncode != 0:
    print(res.stderr)
    raise SystemExit(res.returncode)

In [ ]:
regions_bed_filename = os.path.join(consensus_peak_path)
tss_bed_filename = os.path.join(out_dir, "qc", "tss.bed")

In [ ]:
from pycisTopic.plotting.qc_plot import plot_sample_stats, plot_barcode_stats
import matplotlib.pyplot as plt

In [ ]:
for sample_id in fragments_dict:
    fig = plot_sample_stats(
        sample_id = sample_id,
        pycistopic_qc_output_dir = "outs/qc"
    )

In [ ]:
from pycisTopic.qc import get_barcodes_passing_qc_for_sample
sample_id_to_barcodes_passing_filters = {}
sample_id_to_thresholds = {}
for sample_id in fragments_dict:
    (
        sample_id_to_barcodes_passing_filters[sample_id],
        sample_id_to_thresholds[sample_id]
    ) = get_barcodes_passing_qc_for_sample(
            sample_id = sample_id,
            pycistopic_qc_output_dir = "outs/qc",
            unique_fragments_threshold = 0, # None, use automatic thresholding
            tss_enrichment_threshold = 0, # None, use automatic thresholding
            frip_threshold = 0,
            use_automatic_thresholds = True,
    )

In [ ]:
for sample_id in fragments_dict:
    fig = plot_barcode_stats(
        sample_id = sample_id,
        pycistopic_qc_output_dir = "outs/qc",
        bc_passing_filters = sample_id_to_barcodes_passing_filters[sample_id],
        detailed_title = False,
        **sample_id_to_thresholds[sample_id]
    )

# Creating a cisTopic object

In [ ]:
path_to_regions = os.path.join(consensus_peak_path)
pycistopic_qc_output_dir = "outs/qc"

from pycisTopic.cistopic_class import create_cistopic_object_from_fragments
import polars as pl

cistopic_obj_list = []
for sample_id in fragments_dict:
    sample_metrics = pl.read_parquet(
        os.path.join(pycistopic_qc_output_dir, f'{sample_id}.fragments_stats_per_cb.parquet')
    ).to_pandas().set_index("CB").loc[ sample_id_to_barcodes_passing_filters[sample_id] ]
    cistopic_obj = create_cistopic_object_from_fragments(
        path_to_fragments = fragments_dict[sample_id],
        path_to_regions = path_to_regions,
        path_to_blacklist = path_to_blacklist,
        metrics = sample_metrics,
        valid_bc = sample_id_to_barcodes_passing_filters[sample_id],
        n_cpu = 1,
        project = sample_id,
        split_pattern = '__'
    )
    cistopic_obj_list.append(cistopic_obj)

In [ ]:
cistopic_obj = cistopic_obj_list[0]
print(cistopic_obj)

In [ ]:
import pickle
pickle.dump(
    cistopic_obj,
    open(os.path.join(out_dir, "cistopic_obj.pkl"), "wb")
)

# Adding metadata to a cisTopic object

In [ ]:
#### MUltiome MODE
import scanpy as sc
import pandas as pd
adata = sc.read_h5ad(f"{data_path}/data/results/h5ad.h5ad")
cell_data = adata.obs
cell_data['celltype'] = cell_data[group_col].astype(str)
cell_data['celltype'] = cell_data['celltype'].str.replace(r'[ /]', '_', regex=True)
cell_data['sample_id'] = ID_use
#cell_data['barcode'] = cell_data.index.astype(str) + '-' + ID_use
#cell_data.index = cell_data['barcode']
print(cell_data.shape)
uniques = cell_data['celltype'].unique()
print(uniques)
cell_data
print(cell_data['celltype'].value_counts())

In [ ]:
cistopic_obj.add_cell_data(cell_data, split_pattern='__')
pickle.dump(
    cistopic_obj,
    open(os.path.join(out_dir, "cistopic_obj.pkl"), "wb")
)

In [ ]:
cistopic_obj.cell_data

# Run Models

In [ ]:
import subprocess
import tempfile

In [ ]:
# ---- Command block ----
cmd = f"""
module load jdk/1.8.0_40
pycistopic topic_modeling mallet --input outs/cistopic_obj.pkl --output outs/models.pkl --topics 5 10 15 20 25 30 35 40 --parallel 20 --keep True -b Mallet-202108/bin/mallet
"""

# ---- Run everything in one bash shell ----
res = subprocess.run(["bash", "-lc", cmd], text=True, capture_output=True)

print(res.stdout)
if res.returncode != 0:
    print(res.stderr)
    raise SystemExit(res.returncode)

# Model selection

In [ ]:
from pycisTopic.lda_models import evaluate_models
import pickle
models = pickle.load(open(os.path.join(out_dir, "models.pkl"), "rb"))

In [ ]:
model = evaluate_models(
    models,
    select_model = None,
    return_model = True,
    figsize = (12,5)
)

In [ ]:
cistopic_obj = pickle.load(
    open(os.path.join(out_dir, "cistopic_obj.pkl"), "rb")
)

In [ ]:
cistopic_obj.add_LDA_model(model)

In [ ]:
pickle.dump(
    cistopic_obj,
    open(os.path.join(out_dir, "cistopic_obj.pkl"), "wb")
)

# Clustering and visualization

In [ ]:
from pycisTopic.clust_vis import (
    find_clusters,
    run_umap,
    run_tsne,
    plot_metadata,
    plot_topic,
    cell_topic_heatmap
)

In [ ]:
cistopic_obj = pickle.load(
    open(os.path.join(out_dir, "cistopic_obj.pkl"), "rb")
)

In [ ]:
find_clusters(
    cistopic_obj,
    target  = 'cell',
    k = 10,
    res = [0.6, 1.2, 3],
    prefix = 'pycisTopic_',
    scale = True,
    split_pattern = '__'
)

In [ ]:
run_umap(
    cistopic_obj,
    target  = 'cell', scale=True)

In [ ]:
run_tsne(
    cistopic_obj,
    target  = 'cell', scale=True)

In [ ]:
#cluster the cells based on the topics, see if there could be any subtypes
plot_metadata(
    cistopic_obj,
    reduction_name='UMAP',
    variables=['celltype', 'pycisTopic_leiden_10_0.6', 'pycisTopic_leiden_10_1.2', 'pycisTopic_leiden_10_3'],
    target='cell', num_columns=4,
    text_size=10,
    dot_size=5)

In [ ]:
# plot continuous values.
plot_metadata(
    cistopic_obj,
    reduction_name='UMAP',
    variables=['log10_unique_fragments_count', 'tss_enrichment',  'fraction_of_fragments_in_peaks'],
    target='cell', num_columns=4,
    text_size=10,
    dot_size=5)

In [ ]:
#visualize the cell-topic contributions.
plot_topic(
    cistopic_obj,
    reduction_name = 'UMAP',
    target = 'cell',
    num_columns=5
)

In [ ]:
# draw a heatmap with the topic contributions (and annotations)
cell_topic_heatmap(
    cistopic_obj,
    variables=['celltype'],
    scale=False,
    legend_loc_x=1.0,
    legend_loc_y=-1.2,
    legend_dist_y=-1,
    figsize=(30, 20),
    color_dictionary={}
)

In [ ]:
pickle.dump(
    cistopic_obj,
    open(os.path.join(out_dir, "cistopic_obj.pkl"), "wb")
)

# Topic binarization & QC

In [ ]:
from pycisTopic.topic_binarization import binarize_topics

In [ ]:
cistopic_obj = pickle.load(
    open(os.path.join(out_dir, "cistopic_obj.pkl"), "rb")
)

In [ ]:
region_bin_topics_top_3k = binarize_topics(
    cistopic_obj, method='ntop', ntop = 3_000,
    plot=True, num_columns=5
)

In [ ]:
region_bin_topics_otsu = binarize_topics(
    cistopic_obj, method='otsu',
    plot=True, num_columns=5
)

In [ ]:
binarized_cell_topic = binarize_topics(
    cistopic_obj,
    target='cell',
    method='li',
    plot=True,
    num_columns=5, nbins=100)

In [ ]:
binarized_cell_topic['Topic1']

In [ ]:
from pycisTopic.topic_qc import compute_topic_metrics, plot_topic_qc, topic_annotation
import matplotlib.pyplot as plt
from pycisTopic.utils import fig2img

In [ ]:
topic_qc_metrics = compute_topic_metrics(cistopic_obj)

In [ ]:
fig_dict={}
fig_dict['CoherenceVSAssignments']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Log10_Assignments', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['AssignmentsVSCells_in_bin']=plot_topic_qc(topic_qc_metrics, var_x='Log10_Assignments', var_y='Cells_in_binarized_topic', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['CoherenceVSCells_in_bin']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Cells_in_binarized_topic', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['CoherenceVSRegions_in_bin']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Regions_in_binarized_topic', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['CoherenceVSMarginal_dist']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Marginal_topic_dist', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['CoherenceVSGini_index']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Gini_index', var_color='Gini_index', plot=False, return_fig=True)

In [ ]:
# Plot topic stats in one figure
fig=plt.figure(figsize=(40, 43))
i = 1
for fig_ in fig_dict.keys():
    plt.subplot(2, 3, i)
    img = fig2img(fig_dict[fig_]) #To convert figures to png to plot together, see .utils.py. This converts the figure to png.
    plt.imshow(img)
    plt.axis('off')
    i += 1
plt.subplots_adjust(wspace=0, hspace=-0.70)
plt.show()

In [ ]:
topic_annot = topic_annotation(
    cistopic_obj,
    annot_var='celltype',
    binarized_cell_topic=binarized_cell_topic,
    general_topic_thr = 0.2
)

In [ ]:
topic_annot

# Differentially Accessible Regions (DARs)

In [ ]:
from pycisTopic.diff_features import (
    impute_accessibility,
    normalize_scores,
    find_highly_variable_features,
    find_diff_features
)
import numpy as np

In [ ]:
imputed_acc_obj = impute_accessibility(
    cistopic_obj,
    selected_cells=None,
    selected_regions=None,
    scale_factor=10**6
)

In [ ]:
normalized_imputed_acc_obj = normalize_scores(imputed_acc_obj, scale_factor=10**4)

In [ ]:
variable_regions = find_highly_variable_features(
    normalized_imputed_acc_obj,
    min_disp = 0.05,
    min_mean = 0.0125,
    max_mean = 3,
    max_disp = np.inf,
    n_bins=20,
    n_top_features=None,
    plot=True
)

In [ ]:
len(variable_regions)

In [ ]:
markers_dict= find_diff_features(
    cistopic_obj,
    imputed_acc_obj,
    variable='celltype',
    var_features=variable_regions,
    contrasts=None,
    adjpval_thr=0.1,
    log2fc_thr=np.log2(1.2),
    n_cpu=5,
    _temp_dir=temp_dir,
    split_pattern = '-'
)

In [ ]:
from pycisTopic.clust_vis import plot_imputed_features

In [ ]:
markers_dict

In [ ]:
plot_imputed_features(
    cistopic_obj,
    reduction_name='UMAP',
    imputed_data=imputed_acc_obj,
    features=[markers_dict[x].index.tolist()[0] for x in cistopic_obj.cell_data['celltype'].unique()],
    scale=False,
    num_columns=4
)

In [ ]:
print("Number of DARs found:")
print("---------------------")
for x in markers_dict:
    print(f"  {x}: {len(markers_dict[x])}")

# Save Regions

In [ ]:
os.makedirs(os.path.join(out_dir, "region_sets"), exist_ok = True)
os.makedirs(os.path.join(out_dir, "region_sets", "Topics_otsu"), exist_ok = True)
os.makedirs(os.path.join(out_dir, "region_sets", "Topics_top_3k"), exist_ok = True)
os.makedirs(os.path.join(out_dir, "region_sets", "DARs_cell_type"), exist_ok = True)

In [ ]:
from pycisTopic.utils import region_names_to_coordinates

In [ ]:
for topic in region_bin_topics_otsu:
    region_names_to_coordinates(
        region_bin_topics_otsu[topic].index
    ).sort_values(
        ["Chromosome", "Start", "End"]
    ).to_csv(
        os.path.join(out_dir, "region_sets", "Topics_otsu", f"{topic}.bed"),
        sep = "\t",
        header = False, index = False
    )

In [ ]:
for topic in region_bin_topics_top_3k:
    region_names_to_coordinates(
        region_bin_topics_top_3k[topic].index
    ).sort_values(
        ["Chromosome", "Start", "End"]
    ).to_csv(
        os.path.join(out_dir, "region_sets", "Topics_top_3k", f"{topic}.bed"),
        sep = "\t",
        header = False, index = False
    )

In [ ]:
for cell_type in markers_dict:
    region_names_to_coordinates(
        markers_dict[cell_type].index
    ).sort_values(
        ["Chromosome", "Start", "End"]
    ).to_csv(
        os.path.join(out_dir, "region_sets", "DARs_cell_type", f"{cell_type}.bed"),
        sep = "\t",
        header = False, index = False
    )

In [ ]:
##CHECK OVERLAPS WITH DATABASE, MUST BE > 0

import pandas as pd
from pyranges import PyRanges
import os

# Load cistarget db
df = pd.read_feather(db_path)

regions = pd.Series(df.columns[:-1])
db_coords = regions.str.extract(r"(chr[\w]+):(\d+)-(\d+)")
db_coords.columns = ["Chromosome", "Start", "End"]
db_coords["Start"] = db_coords["Start"].astype(int)
db_coords["End"] = db_coords["End"].astype(int)
db_gr = PyRanges(db_coords)

# Check each .bed file in each folder
bed_folders = [
    os.path.join(out_dir, "region_sets", "Topics_otsu"),
    os.path.join(out_dir, "region_sets", "Topics_top_3k"),
    os.path.join(out_dir, "region_sets", "DARs_cell_type")
]

for bed_folder in bed_folders:
    for fname in os.listdir(bed_folder):
        if not fname.endswith(".bed"):
            continue
        bed_path = os.path.join(bed_folder, fname)
        bed = pd.read_csv(bed_path, sep="\t", header=None, names=["Chromosome", "Start", "End"])
        bed_gr = PyRanges(bed)
        overlap = bed_gr.overlap(db_gr)
        print(f"{fname}: {len(overlap)} overlaps")

# Single Cell RNA Preprocessing

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
import os
import matplotlib as plt
import scanpy as sc
sc.settings.set_figure_params(dpi=80, frameon=False, figsize=(5, 5), facecolor='white')

work_dir = 'outs'
ID = ID_use

#make a directory for to store the processed scRNA-seq data.
if not os.path.exists(os.path.join(work_dir, 'scRNA')):
    os.makedirs(os.path.join(work_dir, 'scRNA'))

In [ ]:
adata = sc.read_h5ad(f"{data_path}/data/results/h5ad.h5ad")
adata.var_names_make_unique()
adata

In [ ]:
adata.raw = adata
adata.raw

In [ ]:
print(adata.raw.X.max())
print(adata.X.max())
print(adata.raw.X.mean())
print(adata.X.mean())

In [ ]:
#adata.obs['celltype'] = adata.obs['combined_celltypes']
adata.obs['celltype'] = adata.obs[group_col].str.replace(r'[ /]', '_', regex=True)
adata.obs['sample_id'] = ID

In [ ]:
adata.raw = adata
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
#adata = adata[:, adata.var.highly_variable]
#sc.pp.scale(adata, max_value=10)

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')
sc.pl.pca_variance_ratio(adata, log=True)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=10)
sc.tl.umap(adata)

In [ ]:
fig = sc.pl.umap(adata, color='celltype', legend_loc='on data', return_fig=True, legend_fontsize=8)

In [ ]:
fig = sc.pl.umap(adata,color = 'celltype', return_fig = True)

In [ ]:
#fig = sc.pl.umap(adata,color = 'Group', return_fig = True, legend_loc='on data', legend_fontsize=8)

In [ ]:
print(adata.raw.X.max())
print(adata.X.max())
print(adata.raw.X.mean())
print(adata.X.mean())

In [ ]:
adata.__dict__['_raw'].__dict__['_var']

In [ ]:
#adata.__dict__['_raw'].__dict__['_var'] = adata.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})
adata.write(os.path.join(work_dir, 'scRNA/adata.h5ad'), compression='gzip')